<a href="https://colab.research.google.com/github/Bast-aleja/Marketing_1/blob/main/C%C3%B3digo_sistema_de_recomendaci%C3%B3n_de_contenido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Modelo de recomendación basados en contenido:***

En esta sección se realizan dos sistemas basados en contenido, con el fin de  ayudar a los usuarios a identificar  peliculas que sean de su interés o preferencia.

In [278]:
!pip install scikit-surprise #Instalación de este paquete. 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


***MODELOS DE CONTENIDO***

In [279]:
#Importación de librerias:
# Además de las librerias basicas, es necesario recurrir a las siguientes librerias:
from sklearn.preprocessing import MinMaxScaler # Ayuda a escalar el año de la pelicula y de este modo lograr que no se encuentre en una escala tan diferente a los dummies.
from ipywidgets import interact # Se utiliza para crear graficos interactivos y así poder facilitar el análisis. 

In [280]:
#Paquete para sistema basado en contenido #
from sklearn import neighbors
 #  El principio detrás de los métodos del vecino más cercano es encontrar un número predefinido de muestras de entrenamiento más 
 #  cercanas en distancia al nuevo punto y predecir la etiqueta a partir de ellas. 

In [281]:
movies.columns # Observamos los nombres de todas las columnas de la base de datos movies.

Index(['movieid', 'title', '(no genres listed)', 'action', 'adventure',
       'animation', 'children', 'comedy', 'crime', 'documentary', 'drama',
       'fantasy', 'film-noir', 'horror', 'imax', 'musical', 'mystery',
       'romance', 'sci-fi', 'thriller', 'war', 'western', 'year'],
      dtype='object')

***Sistema de recomendación basado en contenido KNN***

In [282]:
movies['year']=movies.year.astype('int') # Convertimos el tipo de dato de la variable "year", de string a int (entero), sin embargo, esta columna no se utilizará.

In [283]:
# Escalar para que año esté en el mismo rango ( de 0 a 1), sin embargo, esta columna no se utilizará. 
sc=MinMaxScaler()
movies[["year"]]=sc.fit_transform(movies[['year']])

In [284]:
#Eliminamos las filas que no se van a utilizar y dejamos las que vamos a utilizar para calcular la cercania entre diferentes peliculas. 
movies_dum1=movies.drop(columns=['title','movieid','year']) 

#Que una pelicula coincida con otra y se adapte a los gustos del usuario, no va a depender de que tengan el mismo titulo o se hayan filmado el mismo año
#por ende, decidimos eliminar 'title','movieid','year', debido a que estas columnas no generan mayor información que permita caracterizar  y 
#crear un conjunto de peliculas con  propiedades similares y así  poder recomendarlas al usuario. 
 

In [285]:
# Entrenar modelo
model = neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')# En n_neighbors , colocamos el número de vecinos a usar por defecto para
                                                                   # las consultas, dicho de otr forma, es la cantidad de 
                                                                   # peliculas que se recomendaran al usuario. En cuento a la métrica, se utilizará 
                                                                   #'cosine' que consiste en un producto punto que compara la similitud entre dos vectores. 
model.fit(movies_dum1) # Se ajusta a los datos.
dist, idlist = model.kneighbors(movies_dum1) # 'dist': Devuelve la distancia en el primer vector 
                                             # 'idlisten': Fila de los libros recomendados.  


In [286]:
# Encontramos la distacia de los vectores y la lista de libros recomendados, teniendo en cuenta dicha distancia. 
distancias=pd.DataFrame(dist) 
id_list=pd.DataFrame(idlist)

In [287]:
# Interactivo para mostrar las recomendaciones
def MovieRecommender(movie_name = list(movies['title'].value_counts().index)):
    movie_list_name = []
    movie_id = movies[movies['title'] == movie_name].index 
    movie_id = movie_id[0]
    for newid in idlist[movie_id]:
        movie_list_name.append(movies.loc[newid].title)
    return movie_list_name

print(interact(MovieRecommender)) #Imprimir las recomendaciones con el interactivo. Se mostrará una pelicula y a partir de ella, varias similares. 


interactive(children=(Dropdown(description='movie_name', options=('Hamlet', 'Jane Eyre', 'Misérables, Les', 'C…

<function MovieRecommender at 0x7f811b5f3710>


***Sistema de recomendación basado en contenido de usuario***

Este modelo, toma en cuenta las peliculas que ha visto el usuario y a partir de ese historial logra predecir los posibes gustos del usuario y así establecer una sugerencia de peliculas acordes con dichos gustos. 

In [288]:
userid = 1 #definición de usuario

In [289]:
usuarios=pd.read_sql('select distinct (userId) as user_id from ratings',conn) #Lectura de la lista de usuarios

In [290]:
ratings=pd.read_sql('select * from ratings where userId=:user',conn, params={'user':1}) #Tabla de reatings, en donde estan las calificaciones y se encuentra
                                                                                        #filtrada por un usuario, que en en este preciso caso es el 1.

In [291]:
movies_dum2 = movies

In [292]:
user_id = 1
    
ratings=pd.read_sql('select * from ratings where userId=:user',conn, params={'user':user_id}) #Traer las calificaciones, filtradas por el usuario
l_movie_r=ratings['movieId'].to_numpy() # Para poder filtrar los datos

movies_r=movies_dum2[movies_dum2['movieid'].isin(l_movie_r)] # Se crea 'movies_r a partir de la tabla anteriormente convertida a dummies más la variable movieid. 
                                                             # y se filtrará unicamente la lista de peliculas que ha visto el usuario. 
movies_r=movies_r.drop(columns=['movieid','title']) #Se eliminan las columnas 'movieid' y 'title'
movies_r["indice"]=1 # Se agrega una constante para usar group by y que quede en formato pandas tabla de centroide
centroide=movies_r.groupby("indice").mean() #Centroide de los libros que leyó el usuario ( representación de las películas que le gustan al usuario) 

movies_nr=movies_dum2[~movies_dum2['movieid'].isin(l_movie_r)] #Base de datos de las películas  que no ha visto el usuario.  
movies_nr=movies_nr.drop(columns=['movieid','title']) #Eliminar 'movieid' y 'title'
model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')  # En n_neighbors , colocamos el número de vecinos a usar por defecto para
                                                                   # las consultas, dicho de otra forma, es la cantidad de peliculas que se recomendaran
                                                                   # al usuario, que en este caso son 11 películas. En cuento a la métrica, se utilizará 
                                                                   #'cosine' que consiste en un producto punto que compara la similitud entre dos vectores.
model.fit(movies_nr) #Entrenamiento del modelo para las películas que no leyó el usuario. 
dist, idlist = model.kneighbors(centroide) #Partiendo de las películas que el usuario no ha visto, se buscan las que están más cerca del centroide
                                           # y estas son las que el modelo le recomendará al usuario. 

ids=idlist[0] # Se ke quita una dimensión a la lista. 
recomend_m=movies.loc[ids][['title','movieid']] #Peliculas recomendadas que  el usuario aún no ha visto. 
vistas=movies[movies['movieid'].isin(l_movie_r)][['title','movieid']] #Base de datos de las películas que el usuario ya ha visto. 

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:566: FutureWarning:

Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.



In [293]:
vistas

,title,movieid
0,Toy Story,1
2,Grumpier Old Men,3
5,Heat,6
43,Seven (a.k.a. Se7en),47
46,"Usual Suspects, The",50
...,...,...
2802,Shaft,3744
2836,X-Men,3793
2847,What About Bob?,3809
2991,Transformers: The Movie,4006


In [294]:
# Interactivo para mostrar las recomendaciones
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)): 
    
    ratings=pd.read_sql('select * from ratings where userId=:user',conn, params={'user':user_id}) #Traer las calificaciones, filtradas por el usuario
    l_movie_r=ratings['movieId'].to_numpy()

    movies_r=movies_dum2[movies_dum2['movieid'].isin(l_movie_r)]
    movies_r=movies_r.drop(columns=['movieid','title'])
    movies_r["indice"]=1 # para usar group by y que quede en formato pandas tabla de centroide
    centroide=movies_r.groupby("indice").mean()

    movies_nr=movies_dum2[~movies_dum2['movieid'].isin(l_movie_r)]
    movies_nr=movies_nr.drop(columns=['movieid','title'])
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0]
    recomend_m=movies.loc[ids][['title','movieid']]
    vistas=movies[movies['movieid'].isin(l_movie_r)][['title','movieid']]
    
    return recomend_m

user_id = 2
recomendar(user_id)


print(interact(recomendar))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:566: FutureWarning:

Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.



interactive(children=(Dropdown(description='user_id', options=(1, 410, 403, 404, 405, 406, 407, 408, 409, 411,…

<function recomendar at 0x7f811a4cc8c0>
